In [ ]:
import os
import numpy as np
import psana as ps
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from lv17analysis import epix, lv17data
from scipy.optimize import curve_fit


def gaussian(x, mu, sigma):
    # if amplitude is None:
    amplitude = 1 / ( sigma * np.sqrt(2*np.pi) )
    return amplitude * np.exp( -0.5 * ((x - mu) / sigma)**2 )


def rms(x, axis=None):
    return np.sqrt(np.mean(x**2, axis=axis))


def nanrms(x, axis=None):
    return np.sqrt(np.nanmean(x**2, axis=axis))


def filter_and_flatten(data, dmin=None, dmax=None, exclude_zero=True):

    if dmin is None:
        dmin = np.min(data)
    if dmax is None:
        dmax = np.max(data)
    datos = np.copy(data).flatten()
    datos = datos[datos >= dmin]
    datos = datos[datos <= dmax]
    datos = datos[datos != 0]

    return datos


def fit_gaussian_zero_mode(data, bins=101):
    '''
    Fits gaussian into pixel value histogram of given image.
    Returns:
        mu = mean
        sigma = standard deviation
        rms = root mean square

    Anatoli Ulmer, 2022
    '''
    if len(bins) == 1:
        n_bins = np.copy(bins)
        bins = np.linspace(np.min(data), np.max(data), n_bins)

    hist, bins = np.histogram(data, bins=bins, density=True)
    mu, sigma, covariance = fit_gaussian(bins, hist)
    rms_val = nanrms(data)

    return mu, sigma, rms_val


def fit_gaussian(x, y):
    '''
    Fits gaussian into pixel value histogram of given image.
    Returns:
        mu = mean
        sigma = standard deviation
        rms = root mean square

    Anatoli Ulmer, 2022
    '''
    if len(x) > len(y):
        x = np.copy(bins)
        x = (x[1:] + x[:-1])/2

    fit_parameters, covariance = curve_fit(gaussian, x, y)
    mu, sigma = fit_parameters

    return mu, sigma, covariance


def plot_hist_gauss(ax, data, bins, mu, sigma, rms_val, cm_thresh):
    n, bins, patches = ax.hist(data, bins=bins, density=True, facecolor='green',
                               alpha=0.75, label='pixel histogram')
    plt.yscale('log')
    ylims = plt.ylim()
    plt.vlines(cm_thresh, ylims[0], ylims[1], colors='m', linestyles='dashed',
                label=r"cm thresh$ = {:.3f}$".format(cm_thresh))
    line_plt = plt.plot(bins, gaussian(bins, mu, sigma), 'r:', linewidth=2, label=
                        r"$\sigma = {:.3f}$, rms$ = {:.3f}$".format(sigma, rms_val))
    plt.vlines(photon_energy_kev, ylims[0], ylims[1], colors='r', linestyles='dashed',
                label=r"h$\nu = {:.0f}\,$eV".format(photon_energy_kev*1000))
    plt.vlines(mu, ylims[0], ylims[1], colors='c', linestyles='dashed',
                label=r"$\mu = {:.3f}$".format(mu))

    plt.ylim(ylims)
    plt.xlabel('photon energy / keV')
    plt.ylabel('probability / %')
    # plt.title(r'$\mathrm{Gaussian\ fit:}\ \mu=%.3f,\ \sigma=%.3f,\ \mathrm{rms}=%.3f$' %(mu, sigma, rms_val))
    plt.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=2)
    plt.grid(color='0.8', linestyle='dashed')

    return n, bins, line_plt



In [ ]:
exp_name = 'tmolv1720'
run_list = [296, 439, 437]
tof_dir = '/cds/data/psdm/tmo/tmolv1720/results/shared/tof_run_mean'
detectors = ['gmd', 'xgmd', 'ebeam', 'hsd', 'epix100', 'evr_ch0_delay', 'timing', 'tmo_opal3']

cm_thresh_hv = 0.5
bins_photons = 2
nbins = 101
vrange = [cm_thresh_hv, 100]
cmap = "magma"

img_selector = 'image' # 'raw', 'calib' or 'image'

ds = ps.DataSource(exp=exp_name, run=run_list,
                   detectors=detectors)


for ds_run in ds.runs():
    if type(ds_run) == ps.psexp.null_ds.NullRun:
        sleep(0.1)
        print('Nullrunning! Continuing...')
        continue

    print("run {:.0f}".format(ds_run.runnum))

    epix100_det = ds_run.Detector('epix100')
    # timing_det = ds_run.Detector('evr_ch0_delay') # cluster source delay
    ebeam_det = ds_run.Detector("ebeam")  # electron beam parameters
    gmd_det = ds_run.Detector('gmd')  # gas monitoring detector
    xgmd_det = ds_run.Detector('xgmd')  # gas monitoring detector
    hsd_det = ds_run.Detector('hsd')  # digitizer
    # evr_det = ds_run.Detector('timing') # event codes

    nimgs = 0

    for i, evt in enumerate(ds_run.events()):

        # if nimgs >= np.size(cmaps):
        if nimgs >= 4:
             break

        img_epix = epix.get_img(epix100_det, evt, img_selector=img_selector)

        # photon_energy = ebeam_det.raw.ebeamPhotonEnergy(evt) # photon enerrgy
        photon_energy_kev = lv17data.photon_energy_kev(ds_run.runnum)
        gmd_energy = gmd_det.raw.energy(evt) # pulse energy before attenuation
        xgmd_energy = xgmd_det.raw.energy(evt) # pulse energy after attenen.
        # evt_codes = evr_det.raw.eventcodes(evt) # custom event codes

        # important: always check for missing data
        if img_epix is None: continue
        # if cluster_source_delay is None: cluster_source_delay = np.nan()
        # if photon_energy is None or np.isinf(photon_energy): continue
        if gmd_energy is None: continue
        if xgmd_energy is None: continue
        # if evt_codes is None or not evt_codes[71]: continue



        dmin = -photon_energy_kev*bins_photons
        dmax = photon_energy_kev*bins_photons
        bins = np.linspace(dmin, dmax, nbins)

        # img_masked = epix.masked_img(img_epix)
        
        data_epix = filter_and_flatten(img_epix, dmin=dmin, dmax=dmax)
        mu, sigma, rms_val = fit_gaussian_zero_mode(data_epix, bins=bins)
        cm_thresh = mu + photon_energy_kev * cm_thresh_hv


        ##### 1. PLOT #####
        fig = plt.figure(figsize=(15,3))
        ax = fig.add_subplot(1,3,1)
        n, bins, line_plt = plot_hist_gauss(ax, data=data_epix, bins=bins, mu=mu, sigma=sigma, 
                rms_val=rms_val, cm_thresh=cm_thresh)
        ##### END 1. PLOT #####


        img_cm_corr = np.ma.copy(img_epix)
        # img_cm_corr = epix.cm_correction(img_cm_corr, cm_thresh=cm_thresh)
        img_cm_corr = epix.cm_correction(img_cm_corr, axis=0, cm_thresh=cm_thresh)
        data_cm = filter_and_flatten(img_cm_corr, dmin=dmin, dmax=dmax)
        mu, sigma, rms_val = fit_gaussian_zero_mode(data_cm, bins=bins)
        cm_thresh = mu + photon_energy_kev * cm_thresh_hv


        ##### 2. PLOT #####
        ax = fig.add_subplot(1,3,2)
        n, bins, line_plt = plot_hist_gauss(ax, data=data_cm, bins=bins, mu=mu, sigma=sigma, 
                        rms_val=rms_val, cm_thresh=cm_thresh)
        ##### END 2. PLOT #####


        img_cm_corr2 = np.ma.copy(img_cm_corr)
        img_cm_corr2 = epix.cm_correction(img_cm_corr2, axis=1, cm_thresh=cm_thresh)
        data_cm2 = filter_and_flatten(img_cm_corr2, dmin=dmin, dmax=dmax)
        mu, sigma, rms_val = fit_gaussian_zero_mode(data_cm2, bins=bins)
        cm_thresh = mu + photon_energy_kev * cm_thresh_hv


        ##### 3. PLOT #####
        ax = fig.add_subplot(1,3,3)
        plot_hist_gauss(ax, data=data_cm2, bins=bins, mu=mu, sigma=sigma, 
                        rms_val=rms_val, cm_thresh=cm_thresh)
        ##### 3. PLOT #####

        plt.show()

        nimgs += 1

